In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['ZAP']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('zinc finger antiviral protein', 85),
 ('zymosan activated plasma', 25),
 ('zaprinast', 13),
 ('zc3hav1', 5),
 ('zoster associated pain', 3)]

In [6]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8890)

INFO: [2020-10-02 02:12:40] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.2/bio_ontology.pkl


In [7]:
result = [grounding_map, names, pos_labels]

In [8]:
result

[{'zaprinast': 'MESH:C011145',
  'zc3hav1': 'HGNC:23721',
  'zinc finger antiviral protein': 'HGNC:23721',
  'zoster associated pain': 'MESH:D051474',
  'zymosan activated plasma': 'CHEBI:CHEBI:37671'},
 {'MESH:C011145': 'zaprinast',
  'HGNC:23721': 'ZC3HAV1',
  'MESH:D051474': 'Neuralgia, Postherpetic',
  'CHEBI:CHEBI:37671': '(1->3)-beta-D-glucan'},
 ['CHEBI:CHEBI:37671', 'HGNC:23721', 'MESH:C011145']]

In [9]:
grounding_map, names, pos_labels = [{'zaprinast': 'MESH:C011145',
  'zc3hav1': 'HGNC:23721',
  'zinc finger antiviral protein': 'HGNC:23721',
  'zoster associated pain': 'MESH:D051474',
  'zymosan activated plasma': 'CHEBI:CHEBI:37671'},
 {'MESH:C011145': 'zaprinast',
  'HGNC:23721': 'ZC3HAV1',
  'MESH:D051474': 'Neuralgia, Postherpetic',
  'CHEBI:CHEBI:37671': '(1->3)-beta-D-glucan'},
 ['CHEBI:CHEBI:37671', 'HGNC:23721', 'MESH:C011145']]

In [10]:
excluded_longforms = []

In [11]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [12]:
additional_entities = {}

In [13]:
unambiguous_agent_texts = {}

In [14]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [19]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [20]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [21]:
intersection1

[('HGNC:1848', 'HGNC:1848', 86)]

In [22]:
intersection2

[('MESH:D018893', 'HGNC:1848', 0),
 ('HGNC:1848', 'HGNC:1848', 1),
 ('ungrounded', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:64198', 'HGNC:1848', 0),
 ('MESH:D019164', 'HGNC:1848', 0),
 ('MESH:C059416', 'HGNC:1848', 0),
 ('CHEBI:CHEBI:15710', 'HGNC:1848', 0)]

[('HGNC:7321', ['Musculin', 'musculin'])]

In [23]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(new_pmids)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items()])

In [15]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-02 02:20:49] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-02 02:20:52] /adeft/PythonRepos/adeft/adeft/modeling/classify.py - Best f1 score of 0.9623324258923169 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [25]:
classifier.stats

{'label_distribution': {'MESH:D018893': 1259,
  'HGNC:1848': 86,
  'ungrounded': 17,
  'CHEBI:CHEBI:64198': 8,
  'MESH:D019164': 18,
  'MESH:C059416': 3,
  'CHEBI:CHEBI:15710': 2},
 'f1': {'mean': 0.991104, 'std': 0.001395},
 'precision': {'mean': 0.985357, 'std': 0.002279},
 'recall': {'mean': 0.997026, 'std': 0.002782},
 'CHEBI:CHEBI:15710': {'f1': {'mean': 0.2, 'std': 0.4},
  'pr': {'mean': 0.2, 'std': 0.4},
  'rc': {'mean': 0.2, 'std': 0.4}},
 'HGNC:1848': {'f1': {'mean': 0.982165, 'std': 0.014576},
  'pr': {'mean': 0.965359, 'std': 0.028309},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D018893': {'f1': {'mean': 0.991725, 'std': 0.001462},
  'pr': {'mean': 0.999206, 'std': 0.001587},
  'rc': {'mean': 0.984359, 'std': 0.002417}},
 'CHEBI:CHEBI:64198': {'f1': {'mean': 0.466667, 'std': 0.4},
  'pr': {'mean': 0.4, 'std': 0.374166},
  'rc': {'mean': 0.6, 'std': 0.489898}},
 'ungrounded': {'f1': {'mean': 0.67, 'std': 0.345832},
  'pr': {'mean': 0.616667, 'std': 0.331662},
  'rc': {'mean':

In [16]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [18]:
disamb.dump(model_name, results_path)

In [17]:
print(disamb.info())

Disambiguation model for ZAP

Produces the disambiguations:
	(1->3)-beta-D-glucan*	CHEBI:CHEBI:37671
	Neuralgia, Postherpetic	MESH:D051474
	ZC3HAV1*	HGNC:23721
	zaprinast*	MESH:C011145

Class level metrics:
--------------------
Grounding              	Count	F1     
                ZC3HAV1*	63	0.97131
   (1->3)-beta-D-glucan*	25	   0.95
              zaprinast*	 9	0.93333
Neuralgia, Postherpetic	 2	    0.2

Weighted Metrics:
-----------------
	F1 score:	0.96233
	Precision:	0.96591
	Recall:		0.97

* Positive labels
See Docstring for explanation



In [19]:
model_to_s3(disamb)